In [2]:
import sqlParser
import os
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
with open("sample.sql", "r", encoding="utf8") as sql_file:
        table_info = sqlParser.parse_sql(sql_file.read().strip())
        sqlParser.generate_er_xml(table_info)

In [15]:
Client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE")
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    message = [
        {
            "role": "system",
            "content": 'For a given ER design text or image, use json to describe entities, attributes, and relationships between entities (one-to-one, one-to-many, or many-to-many). For example: {"ERD": [{"entity": "Entity1","attributes": ["Attr1", "Attr2"],"relationship":[{"type": "one-to-one","entity": "Entity2"}]},{"entity": "Entity2","attributes": ["Attr3", "Attr4"],"relationship": []}]}'
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]

    response = Client.chat.completions.create(
        messages=message,
        model=model,
    )

    return response.choices[0].message.content

In [16]:
prompt = """
- 学生(Student)
    - 属性: 学号(ID)、姓名(Name)、年龄(Age)
- 课程(Course)
    - 属性: 课程编号(CourseID)、课程名称(CourseName)
- 选课(Enrollment)
    - 属性: 成绩(Grade)
    - 外键: 学号(StudentID)、课程编号(CourseID)
"""
out = get_completion(prompt)
print(out)

{"ERD": [{"entity": "Student","attributes": ["ID", "Name", "Age"],"relationship":[{"type": "one-to-many","entity": "Enrollment"},{"type": "one-to-one","entity": "Course"}]},{"entity": "Course","attributes": ["CourseID", "CourseName"],"relationship":[{"type": "one-to-many","entity": "Enrollment"}]},{"entity": "Enrollment","attributes": ["Grade"],"relationship":[{"type": "many-to-one","entity": "Student"},{"type": "many-to-one","entity": "Course"}]}]}
